# Modélisation de la consommation et de la production d'un foyer équipé d'un photovoltaique, d'une éolienne et d'une batterie

Ce notebook permet de reproduire les expériences du papier :

TODO:
- inclure la prediction de la consommation
- inclure le temps de calcul
- update sources des données
- IoT peut foncitonner en temps reel mais limitation par la resolution des donnees obtenues
    
https://www.meteoblue.com/fr/historyplus
    
## Estimation de la consommation
On estime que le foyer a une consommation électrique annuelle de 20000 kWh[1]. Les données proviennent du gouvernement français[2]. Les heures creuses/pleines sont prises pour Cachan.

[1] http://www.monenergie.net/consommation-moyenne-d-electricite-d-une-maison.php "Consommation moyenne d'une maison"

[2] https://www.data.gouv.fr/fr/datasets/bilan-electrique-au-pas-demi-heure/ "Dataset de consommation"

## Prix de l'électricité
http://www.kelwatt.fr/guide/prix-par-mois#base

http://www.jechange.fr/energie/electricite/guides/prix-electricite-kwh-2435
- Abonnement annuel : 117,50 €
- Evolution des prix sur 2013 en suivant l'historique de :
http://www.kelwatt.fr/prix/tarif-bleu-edf#trouver-moins-cher-tarif-bleu
- prix du kWh: (heures pleines, heures creuses) = 0.129, 0.105


Rachat du photovoltaïque :
- Intégration au bâti :
    0-9 kWc = 24,29 cts€/kWh (24,64 cts€/kWh le trimestre précédent)
- Intégration simplifiée au bâti :
    0-36 kW = 12,72 cts€/kWh (13,27 cts€/kWh le trimestre précédent)
    36-100 kW = 12,13 cts€/kWh (12,62 cts€/kWh le trimestre précédent)
- Sans intégration, ou hors critères ci-dessus, ou au sol:
    5,65 c€/kWh (5,96 cts€/kwh le trimestre précédent)

Rachat de l'éolien :
- http://www.developpement-durable.gouv.fr/Les-tarifs-d-achat-de-l,12195.html
- prix: 8.2cEUR/kWh


In [10]:
from mpl_toolkits.axes_grid1.parasite_axes import host_subplot
from mpl_toolkits.axisartist.axislines import Axes
from scipy.sparse.csr import csr_matrix
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.svm.classes import NuSVR
from sklearn.ensemble.gradient_boosting import GradientBoostingRegressor
from sklearn.linear_model.stochastic_gradient import SGDClassifier
from sklearn.metrics import hamming_loss


# -*- coding: latin-1 -*-
import locale, time
locale.setlocale(locale.LC_TIME, "fr_FR".encode('ascii'))
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import rc
from __future__ import unicode_literals
import numpy as np
import matplotlib
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.unicode'] = True
rc('font',**{'family':'serif','serif':['Times']})
%matplotlib osx

# données de consommation
df=pd.read_csv('dataset/consommation_suisse_2015.csv', sep=",")
df["consommation"].head()
df["date"]= df["time"].str.split(" - ").str[0]
df["date"]= pd.to_datetime(df["date"])
df = df.set_index(['date'])
df = df[~df.index.duplicated()]
df = df.sort_index()
hour = df.index.hour
month = df.index.month
HC = ((hour >= 2) & (hour < 7)) | ((hour >= 14) & (hour < 17))
HP =  ((hour >= 7) & (hour < 14)) | (hour >= 17) | (hour < 2)
consoPerso = 20000 # consommation annuelle kWh d'un foyer
# 20000 pour un foyer : http://www.monenergie.net/consommation-moyenne-d-electricite-d-une-maison.php
consoTotale   = df["consommation"].sum()/1000 # consommation annuelle en kWh
df["consommation"] =  df["consommation"]*consoPerso/consoTotale

prix_achat = np.array([0.129, 0.105])
# evolution = np.array([2.3, 0, 0, 0 ,0 ,0, 0, 5, 0,0,0,0])
inflation = (1+evolution/100).cumprod()
df["Inflation"] = pd.Series(np.nan, df.index)
df["Inflation"].loc[(df.index.day == 1) & (df.index.hour == 0)] = inflation
df["Inflation"] = df["Inflation"].fillna(method='ffill')
abonnement = 117.5
abonnement_heure = abonnement/365.25/24
prix_hc = prix_achat[1]* df["Inflation"] +abonnement_heure
prix_hp = prix_achat[0]* df["Inflation"] +abonnement_heure
prix = prix_hp
prix.loc[(hour >= 2) & (hour < 7)] = prix_hc[(hour >= 2) & (hour < 7)]
prix.loc[(hour >= 14) & (hour < 17)] = prix_hc[(hour >= 14) & (hour < 17)]
df["depense"] = df["consommation"].copy()
df["depense"].loc[HC] *= prix_hc
df["depense"].loc[HP] *= prix_hp
prix_pv = 0.1272 - abonnement_heure
prix_eolien = 0.082 - abonnement_heure 


# données Bale
bale=pd.read_csv('dataset/bale.csv', sep=";", skiprows=11)
bale["date"]=pd.to_datetime({
        'year': bale["Year"],
        'month': bale["Month"],
        'day': bale["Day"],
        'hour': bale["Hour"],
                    })
bale = bale.set_index(['date'])
bale = bale[(bale.index.year == 2013)]
bale["temperature"] = bale["Temperature  [2 m above gnd]"]
bale["radiation"] = bale["Shortwave Radiation - backwards  [sfc]"]/1000
bale["consommation"] = df['consommation']
bale["depense"] = df["depense"] # depense sans PV
bale["inflation"] = df["Inflation"]
bale['rafale'] = bale['Wind Gust  [10 m above gnd]']
bale['vent'] = bale['Wind speed  [10 m above gnd]']

In [ ]:
def Depense(manque, rachat=prix_eolien):
    depense = pd.Series(0, index=manque.index)
    depense[manque.values < 0] += manque[manque.values < 0]*rachat
    depense[(manque.values > 0)] += manque[(manque.values > 0)]*prix
    return depense.cumsum()

# test:
# manque = pd.Series(np.random.rand(len(bale.index)), index=bale.index)
# Depense(manque,(bale.index.year == 2013)).plot()

## Prédiction de la consommation

In [ ]:
def prediction(day):
    '''At the end of each day, on trained our model with one month previous data'''
    
       {'estimator': GradientBoostingRegressor,
     'tuned_params': {'loss': 'ls'},
     'changing_param': 'n_estimators',
     'changing_param_values': [300, 500, 1000, 1500],
     'complexity_label': 'n_trees',
     'complexity_computer': lambda x: x.n_estimators,
     'data': regression_1month,
     'postfit_hook': lambda x: x,
     'prediction_performance_computer': mean_squared_error,
     'prediction_performance_label': 'MSE',
     'n_samples': 30}
        

    conf['tuned_params'][conf['changing_param']] = param_value
        estimator = conf['estimator'](**conf['tuned_params'])
#         print("Benchmarking %s" % estimator)
        estimator.fit(conf['data']['X_train'], conf['data']['y_train'])
        conf['postfit_hook'](estimator)
        complexity = conf['complexity_computer'](estimator)
        complexities.append(complexity)
        start_time = time.time()
        for _ in range(conf['n_samples']):
            y_pred = estimator.predict(conf['data']['X_test'])
        elapsed_time = (time.time() - start_time) / float(conf['n_samples'])
        prediction_times.append(elapsed_time)
        pred_score = conf['prediction_performance_computer'](
            conf['data']['y_test'], y_pred)
        prediction_powers.append(pred_score)
#         print("Complexity: %d | %s: %.4f | Pred. Time: %fs\n" % (
#             complexity, conf['prediction_performance_label'], pred_score,
#             elapsed_time))
    return prediction_powers, prediction_times, complexities, y_pred


# Lecture des donnés historiques de Bâle en ensoleillement et température :


## Production d'électricité photovoltaïque et éolienne
Thèse O. Gergaud, modèle polynomial
$P_{G,max} = P_1\left[ 1 + P_2 (T - T_{ref}) \right] (P_3 + E_s)$
avec :
- $P_1 = 0.99$ (ou 0.099 pour un panneau)
- $P_2 = -4.7e^{-3} K^{-1}$ dérive en température
- $P_3 = 45$
- $T_{ref} = 24°C$
- $E_s$ ensoleillement dans le plan de panneaux (W/m^2)

Eolienne considérée :
- http://www.fw4sea.com/produits-fr/fiches-techniques/80-fiches-techniques-fr/88-f180-50

In [11]:
P1 =  6 #rendement de 30m2 de panneau solaire
P2 = -4.7e-3 #deviation en temperatrue
P3 = 45e-3 #gain supplémentaire
Tref = 24
NOCT = 43
bale["prod-pv"] = P1*( 1 + P2*(bale["radiation"]* (NOCT-20)/800))*(P3+bale["radiation"])
period = (bale.index.month == 6) & (bale.index.day == 30)

# F180-50
# puissance = np.ones(20)* 50.0
# puissance[:10]= [0,0,1,2.16,5.12,9.5,15.55, 23.52, 33.65, 45.0]
# Modele F100-10
puissance = np.ones(20)* 10.0

# Modele F180-36
puissance = np.ones(20)* 36.0
puissance[:10]= [0,0,0,0.68,2.5, 5.45, 9.85, 15.9, 23.85, 33.9]
plt.figure()
bale['prod-eol'] = np.interp( bale['vent'].values, np.arange(20.0)*3.6, puissance)
ax = plt.plot(bale['vent'], bale['prod-eol'].values, '+')
plt.xlabel("Vitesse du vent (m/s)")
plt.ylabel("Puissance produite (kW)")
plt.show()


plt.figure()
ax = bale[period].plot(y='prod-eol', label="Eolien")
bale[period].plot(y="prod-pv", label="Photovoltaique", ax=ax)
bale[period].plot(y="consommation", label="Consommation", ax=ax)
ax.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=3, mode="expand", borderaxespad=0.)
ax = plt.gca()
ax.yaxis.grid()
plt.xlabel(bale[period].index[0].strftime("%d %b %Y"))
plt.ylabel("kWh")
ticklabels = ['']*len( bale[period].index)
ticklabels[::3] = [item.strftime('%-H') for item in  bale[period].index[::3]]
ax.xaxis.set_major_formatter(matplotlib.ticker.FixedFormatter(ticklabels))
# ax.gcf().autofmt_xdate()



## Loi de commande 
    - Vente: on vend tout, puis on achète selon nos besoins
    - Consommation: on consomme ce qu'on produit, puis on vend les surplus
    - BatterieNormale: on stocke les surplus
    - BatterieAchat: à la fin d'une période d'heures creuses, on achète pour remplir la batterie jusqu'à un certain stade
    - BatterieAchatOpt: à la fin d'une période d'heures creuses, on achète pour remplir la batterie afin ne pas manquer d'électricité lors d'une période d'heures pleines

In [12]:
def Vente(consommation, production, mask, rachat=prix_eolien):
    depense = -production[mask]*rachat
    depense += consommation[mask]*prix[mask]
    return depense.cumsum()

def Consommation(consommation, production, mask, rachat=prix_eolien):
    besoin = (consommation - production)
    return Depense(besoin[mask], rachat)

def BatterieNormale(besoin, cap_batterie, period, rendement=0.9):
    batteriePleine = pd.Series(cap_batterie, index=bale[period].index) 
    stockage = pd.Series(0, index=bale[period].index) 
    manque = pd.Series(0, index=bale[period].index)   # manque  : > 0 si reseau > maison, <= 0 sinon
    stockagePre = 0.0
    for p in bale[period].index:
        m=  bale[period].index == p
        if besoin[m].values[0] > 0:
            manque[m] = max(-stockagePre*rendement+besoin[m].values[0],0)
            stockage[m] = max(stockagePre-besoin[m].values[0]/rendement,0)
        elif besoin[m].values[0] < 0 and -besoin[m].values[0] > (cap_batterie - stockagePre):
                stockage[m]=cap_batterie
                manque[m] = (cap_batterie - stockagePre) -(-besoin[m].values[0]) # < 0 
        elif besoin[m].values[0] < 0 and -besoin[m].values[0] < (cap_batterie - stockagePre):
                manque[m] = 0
                stockage[m] = -besoin[m].values[0] + stockagePre
        stockagePre = stockage[m].values[0]

    return [manque, stockage]

def BatterieAchat(besoin, cap_batterie, period, coeff = 1.0, rendement = 0.9):
    batteriePleine = pd.Series(cap_batterie, index=bale[period].index) 
    stockage = pd.Series(0, index=bale[period].index) 
    manque = pd.Series(0, index=bale[period].index)   # manque  : > 0 si reseau > maison, <= 0 sinon
    stockagePre = 0.0
    for p in bale[period].index:
        m=  bale[period].index == p
        transfert = 0 # > 0 si maison -> batterie, <= 0 sinon
        if besoin[m].values[0] > 0:
            manque[m] = max(-stockagePre*rendement+besoin[m].values[0],0)
            stockage[m] = max(stockagePre-besoin[m].values[0]/rendement,0)
        elif besoin[m].values[0] < 0 and (-besoin[m].values[0] > cap_batterie - stockagePre):
                stockage[m]=cap_batterie
                manque[m] = cap_batterie - stockagePre -(-besoin[m].values[0]) # < 0 
        elif besoin[m].values[0] < 0 and (-besoin[m].values[0] < cap_batterie - stockagePre):
                manque[m] = 0
                stockage[m] = -besoin[m].values[0] + stockagePre
        h = p.hour
        if (h == 16 or h == 6) and (stockage[m].values[0] < cap_batterie*coeff):
            manque[m] += cap_batterie*coeff - stockage[m].values[0] 
            stockage[m] = cap_batterie*coeff 
        stockagePre = stockage[m].values[0]

    return [manque, stockage]

def Anticipation(besoin,cap_batterie, period, rendement = 0.9):
# calcul ce dont il faut acheter pour subvenir aux besoins futurs de la HP
    [manque, stockage] = BatterieNormale(besoin, cap_batterie, period, rendement)
    return manque[manque > 0 ].sum()
    
def BatterieAchatOptimal(besoin, cap_batterie, period, rendement = 0.9):
    batteriePleine = pd.Series(cap_batterie, index=bale[period].index) 
    stockage = pd.Series(0, index=bale[period].index) 
    manque = pd.Series(0, index=bale[period].index)   # manque  : > 0 si reseau > maison, <= 0 sinon
    stockagePre = 0.0
    for p in bale[period].index:
        m=  bale[period].index == p
        transfert = 0 # > 0 si maison -> batterie, <= 0 sinon
        if besoin[m].values[0] > 0:
            manque[m] = max(-stockagePre*rendement+besoin[m].values[0],0)
            stockage[m] = max(stockagePre-besoin[m].values[0]/rendement,0)
        elif besoin[m].values[0] < 0 and (-besoin[m].values[0] > cap_batterie - stockagePre):
                stockage[m]=cap_batterie
                manque[m] = cap_batterie - stockagePre -(-besoin[m].values[0]) # < 0 
        elif besoin[m].values[0] < 0 and (-besoin[m].values[0] < cap_batterie - stockagePre):
                manque[m] = 0
                stockage[m] = -besoin[m].values[0] + stockagePre
        h = p.hour
        if (h == 16):
            periodHP = period & (bale.index >= (p + pd.DateOffset(hours=1)))
            periodHP &= bale.index <= (p + pd.DateOffset(hours=10))
            besoinHP = Anticipation(besoin[periodHP],cap_batterie, periodHP)
            if (besoinHP > stockage[m].values[0] ):
                manque[m] += min(besoinHP, cap_batterie) - stockage[m].values[0] 
                stockage[m] += min(besoinHP, cap_batterie)
        if (h == 6):
            periodHP = period & (bale.index >= (p + pd.DateOffset(hours=1)))
            periodHP &= bale.index <= (p + pd.DateOffset(hours=8))
            besoinHP = Anticipation(besoin[periodHP],cap_batterie, periodHP)
            if (besoinHP > stockage[m].values[0] ):
                manque[m] += min(besoinHP, cap_batterie) - stockage[m].values[0] 
                stockage[m] = min(besoinHP, cap_batterie)

        stockagePre = stockage[m].values[0]

    return [manque, stockage]

## Besoin, manque et stockage sur une journée

In [7]:
plt.figure()
period = (bale.index.month <= 1)
cap_batterie = 7
besoin = (bale['consommation'] - bale['prod-eol'])[period]
speriod = (besoin.index.month == 1) & (besoin.index.day == 5)
[manque, stockage] = BatterieNormale(besoin,  cap_batterie, period)
besoin[speriod].plot(label="Besoin")
manque[speriod].plot(label="Manque")
stockage[speriod].plot(label="Stockage")
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=3, mode="expand", borderaxespad=0.)
ax = plt.gca()
ax.grid(True, which='both')
plt.xlabel("Temps")
plt.ylabel("kWh")
print besoin[speriod].mean()

1.72972966991


In [8]:
plt.figure()
bale['consommation'][period].cumsum().plot(label="Consommation")
bale['prod-pv'][period].cumsum().plot(label="Photovoltaïque")
bale['prod-eol'][period].cumsum().plot(label="Eolien")
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=3, mode="expand", borderaxespad=0.)

plt.xlabel("Temps")
plt.ylabel("kWh")
ax = plt.gca()
ax.yaxis.grid()
plt.show()

## Influence du coefficient de remplissage

In [9]:
period = (bale.index.year == 2013)
cap_batterie = 14
coeff = np.arange(0, 1.1, 0.2)

besoin = (bale['consommation'] - bale['prod-pv'])[period]
coeff_pv = []
for i in coeff:
    print i
    [manque, stockage] = BatterieAchat(besoin, cap_batterie, period, i)
    coeff_pv.append(Depense(manque,prix_pv)[-1])
    
besoin = (bale['consommation'] - bale['prod-eol'])[period]
coeff_eol = []
for i in coeff:
    print i
    [manque, stockage] = BatterieAchat(besoin, cap_batterie, period, i)
    coeff_eol.append(Depense(manque,prix_eolien)[-1])


0.0
0.2
0.4
0.6
0.8
1.0
0.0
0.2
0.4
0.6
0.8
1.0


In [10]:
plt.figure()
plt.plot(coeff, coeff_pv/max(coeff_pv), label="Photovoltaique")
plt.plot(coeff, coeff_eol/max(coeff_eol), label="Eolien")
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.)
ax = plt.gca()
plt.xlabel("Coefficient de remplissage")
plt.ylabel("Facture normalisée")
ax.yaxis.grid()

## Facture avec le photovoltaïque

In [7]:
# period = (bale.index.month == 1) & (bale.index.day == 1)
period = (bale.index.year == 2013)
cap_batterie = 14
besoin = (bale['consommation'] - bale['prod-pv'])[period]
pv = {}
# 1. en vendant systematiquement notre production de photovoltaique
pv['Vente'] = Vente(bale['consommation'], bale['prod-pv'], period, prix_pv)

# 2. On consomme notre production de photovoltaique et on vend ce qu'on ne consomme pas
pv["Consommation"] = Consommation(bale['consommation'], bale['prod-pv'], period, prix_pv)

# 3. On stocke et on n'utilise notre production qu'aux HP
[manqueNormal, stockageN] = BatterieNormale(besoin,  cap_batterie, period)
pv["Normal"] = Depense(manqueNormal,prix_pv)

# 4. On remplit la batterie en fin de HC
[manqueAchat, stockageAchat] = BatterieAchat(besoin, cap_batterie, period, 1.0)
pv["Achat"] = Depense(manqueAchat, prix_pv)

# 5. On remplit la batterie en fin de HC en prévoyant notre besoin futur
[manqueOpt, stockageOpt] = BatterieAchatOptimal(besoin, cap_batterie, period)
pv["Optimal"] = Depense(manqueOpt, prix_pv)


In [13]:
plt.figure()
for name in pv:
    pv[name].plot(label=name)
    print pv[name][-1]
# plt.title("Comparaison des lois de consommation en photovoltaïque")
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=3, mode="expand", borderaxespad=0.)
ax = plt.gca()
plt.xlabel("Temps")
plt.ylabel("Facture (€)")
ax.yaxis.grid()
plt.show()

1228.27360155
1349.32792527
1317.36954639
1382.54764122
1580.19133607


In [9]:
plt.figure()
period = bale.index.year == 2013
cap_batterie = 14
bale['consommation'][period].cumsum().plot(label="Consommation")
bale['prod-pv'][period].cumsum().plot(label="Photovoltaïque")
bale['prod-eol'][period].cumsum().plot(label="Eolien")
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=3, mode="expand", borderaxespad=0.)
plt.xlabel("Temps")
plt.ylabel("kWh")
ax = plt.gca()
ax.yaxis.grid()

## Facture avec l'éolien

In [10]:
# period = (bale.index.month == 1)# & (bale.index.day == 1)
period = (bale.index.year == 2013)
cap_batterie = 14
besoin = (bale['consommation'] - bale['prod-eol'])[period]
eol_dep = {}

# 1. en vendant systematiquement notre production de photovoltaique
eol_dep["Vente"] = Vente(bale['consommation'], bale['prod-eol'], period, prix_eolien)

# 2. On consomme notre production de photovoltaique et on vend ce qu'on ne consomme pas
eol_dep["Consommation"] = Consommation(bale['consommation'], bale['prod-eol'], period, prix_eolien)

# 3. On stocke et on n'utilise notre production qu'aux HP
[manqueNormal, stockageN] = BatterieNormale(besoin,  cap_batterie, period)
eol_dep["Normal"] = Depense(manqueNormal,prix_eolien)

# 4. On remplit la batterie en fin de HC
[manqueAchat, stockageAchat] = BatterieAchat(besoin, cap_batterie, period, 1.0)
eol_dep["Achat"] = Depense(manqueAchat, prix_eolien)

# 5. On remplit la batterie en fin de HC en prévoyant notre besoin futur
[manqueOpt, stockageOpt] = BatterieAchatOptimal(besoin, cap_batterie, period)
eol_dep["Optimal"] = Depense(manqueOpt, prix_eolien)

In [15]:
plt.figure()
for name in eol_dep:
    eol_dep[name].plot(label=name)
    print eol_dep[name][-1]
    

# plt.title("Comparaison des lois de consommation en éolien")
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=3, mode="expand", borderaxespad=0.)
ax = plt.gca()
plt.xlabel("Temps")
plt.ylabel("Facture (€)")
ax.yaxis.grid()

37.6286563575
142.495777793
151.621697037
282.741601979
877.564102391


## Quelle batterie faut-il ?

In [16]:
batteries = range(2,20,2)#[7,10,14]
period = (bale.index.year == 2013)
cap_batterie = 14


# en photovoltaique
pv_batt = {}
besoin = (bale['consommation'] - bale['prod-pv'])[period]
for b in batteries:
    [manqueOpt, stockageOpt] = BatterieAchatOptimal(besoin, b, period)
    pv_batt[b] = Depense(manqueOpt, prix_pv)[-1]


# en eolien
eol_batt = {}
besoin = (bale['consommation'] - bale['prod-eol'])[period]
for b in batteries:
    [manqueOpt, stockageOpt] = BatterieAchatOptimal(besoin, b, period)
    eol_batt[b] = Depense(manqueOpt, prix_eolien)[-1]


# sans production
noprod_batt = {}
besoin = bale['consommation'][period]
for b in batteries:
    [manqueOpt, stockageOpt] = BatterieAchatOptimal(besoin, b, period)
    noprod_batt[b] = Depense(manqueOpt, 0)[-1]

In [19]:
plt.figure()
plt.plot(batteries, pv_batt.values()/max(pv_batt.values()), label="Photovoltaïque")
max_eol = np.max(np.abs(np.array(eol_batt.values())))
eol = (2*max_eol+ np.array(eol_batt.values()))
eol = eol/max(eol)
plt.plot(batteries,eol, label="Eolien")
plt.plot(batteries, noprod_batt.values()/max(noprod_batt.values()), label="Sans production")

plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=3, mode="expand", borderaxespad=0.)
ax = plt.gca()
plt.xlabel("Capacité de batterie")
plt.ylabel("Facture normalisée")
ax.yaxis.grid()

In [7]:
## 

8761
8761
                                                    time  consommation  \
date                                                                     
2015-10-25 02:00:00  25.10.2015 02:00 - 25.10.2015 03:00    436.566993   

                     Inflation     depense  
date                                        
2015-10-25 02:00:00    1.07415  436.566993  
